# Catalog exploration example

In [1]:
import intake
from fresca_catalog import search_catalog_variables, filter_catalog, map_datasets

In [2]:
catalog = intake.open_catalog('full_catalog.yml')
catalog

Catalog
 named datasets: ['all_ctd', 'sfer']
  parameters: ['CATALOG_DIR', 'CATALOG_PATH', 'STORAGE_OPTIONS']

In [3]:
help(search_catalog_variables)

Help on function search_catalog_variables in module fresca_catalog:

search_catalog_variables(catalog, query=None, case_insensitive=True, fuzzy=False) -> List[str]
    Searches the catalog for variables.
    
    Parameters
    ----------
    catalog : Catalog
        The catalog to search.
    query : str, optional
        The query to search for. If not provided, all variables are returned.
    case_insensitive : bool, optional
        Whether to perform a case-insensitive search. Default is True.
    fuzzy : bool, optional
        Whether to perform a fuzzy search. Default is False.
    
    Returns
    -------
    list
        A list of matching variables.



In [4]:
search_catalog_variables(catalog, 'sal')

['sal', 'salinity']

In [5]:
search_catalog_variables(catalog, 'temp')

['temp', 'temperature']

In [6]:
help(filter_catalog)

Help on function filter_catalog in module fresca_catalog:

filter_catalog(catalog: intake.readers.entry.Catalog, entry_names: List[str] = None, variables: List[str] = None, time_range: Tuple[str, str] = None, bbox: Tuple[float, float, float, float] = None) -> intake.readers.entry.Catalog
    Filters a catalog based on the provided criteria.
    
    Parameters
    ----------
    catalog : Catalog
        The catalog to filter.
    entries : list, optional
        The entries to be used for filtering.
    variables : list, optional
        The variables to be used for filtering.
    time_range : list, optional
        The time range to be used for filtering, formatted as [min_time, max_time].
    bbox : list, optional
        The bounding box to be used for filtering, formatted as [min_lon, min_lat, max_lon, max_lat].
    
    Returns
    -------
    Catalog
        The filtered catalog.



In [7]:
my_catalog = filter_catalog(catalog, variables=['temp', 'temperature'])
my_catalog

Catalog
 named datasets: ['all_ctd', 'sfer']

In [8]:
map_datasets(my_catalog)

/home/trevor/miniconda3/envs/fresca/lib/python3.10/site-packages/geopandas/geoseries.py:860: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (dataset)

In [9]:
map_datasets(my_catalog, variables=['temp', 'temperature'])

UnboundLocalError: local variable 'agg_gdf' referenced before assignment

In [ ]:
map_datasets(my_catalog, variables=['temp', 'temperature'], log=True)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (total)

In [ ]:
map_datasets(my_catalog, variables=['temp', 'temperature'], log=True, hex=True)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (total)

In [13]:
my_catalog = filter_catalog(
    catalog,
    variables=['temp', 'temperature'],
    time_range=('2023-12-01T13:23:00Z', '2024-11-17T23:54:00Z'),
    bbox=(-70.0, 20.0, -90.0, 40.0)
)
my_catalog

Catalog
 named datasets: ['sfer']

In [14]:
my_catalog['sfer'].metadata

{'lat_col': 'lat_dec',
 'lon_col': 'lon_dec',
 'maxLatitude': 28.7841,
 'maxLongitude': -80.0442,
 'maxTime': '2024-11-17T00:00:00Z',
 'minLatitude': 24.3952,
 'minLongitude': -85.0221,
 'minTime': '2014-12-01T00:00:00Z',
 'time_col': 'date',
 'variables': ['date',
  'cruise_id',
  'station',
  'event_n',
  'temp',
  'sal',
  'o2_ctd',
  'nh4',
  'no2',
  'no3',
  'no3_no2',
  'po4',
  'si',
  'avg_chl_a',
  'avg_phaeo',
  'lat_dec',
  'lon_dec',
  'station_type']}

In [15]:
df = my_catalog['sfer'].read()
df.head()

,date,cruise_id,station,event_n,temp,sal,o2_ctd,nh4,no2,no3,no3_no2,po4,si,avg_chl_a,avg_phaeo,lat_dec,lon_dec,station_type
0,2014-12-01,WS14335,1,1,1,1,0,1,1,1,1,1,1,1,1,25.6450,-80.1252,F
1,2014-12-01,WS14335,2,1,1,1,0,1,1,1,1,1,1,1,1,25.6415,-80.1042,F
2,2014-12-01,WS14335,3,1,1,1,0,1,1,1,1,1,1,1,1,25.6458,-80.0818,F
3,2014-12-01,WS14335,4,1,1,1,0,1,1,1,1,1,1,1,1,25.1087,-80.3826,F
4,2014-12-01,WS14335,5,1,1,1,0,1,1,1,1,1,1,1,1,25.0906,-80.3591,F


In [12]:
df.to_csv('my_data.csv', index=False)

In [13]:
my_catalog.to_yaml_file('my_catalog.yaml')